In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline 
%config IPCompleter.greedy=True

import collections
import io
import math
import tensorflow as tf
from IPython import display
from sklearn import metrics


## Reading data from file

In [2]:
from collections import Counter

### main dataset ###
data = pd.read_csv("20151219.txt", sep = "\t", header = None)
data.columns = ["conn_len", "service", "src_bytes", "dst_bytes", "conn_numb", "Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "Conn_state", "IDS_detection", "Malware_detection", "Ashula_detection", "attack_flag", "src_IP", "src_port", "dst_IP", "dst_port", "start_time", "proto"]

print(Counter(data["attack_flag"]))

Counter({-1: 149915, 1: 67713})


## Transform to binary

In [3]:
data.loc[:, "attack_flag"] = data.loc[:,"attack_flag"].replace(-2, -1)
data.loc[:, "attack_flag"] = data.loc[:, "attack_flag"] * -1
data.loc[:, "attack_flag"] = data.loc[:,"attack_flag"].replace(-1, 0)

print(Counter(data["attack_flag"]))

Counter({1: 149915, 0: 67713})


In [4]:
# removing NaNs
data = data.dropna()
data.shape

(217628, 24)

# a bit of feature engineering

## port numbers classification

In [5]:
t1 = 1023  # well-known port numbers
t2 = 49151 # registered ports
t3 = 65535 # client ports

def wk(data_row):
    if (data_row["src_port"] <= t1): 
        value = 1
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 0
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 0
    return value

def reg(data_row):
    if (data_row["src_port"] <= t1): 
        value = 0
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 1
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 0
    return value

def cli(data_row):
    if (data_row["src_port"] <= t1): 
        value = 0
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 0
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 1
    return value

data["well_known_src_pool"] = data.apply(wk, axis=1)
data["registered_src_pool"] = data.apply(reg, axis=1)
data["cli_src_pool"] = data.apply(cli, axis=1)

In [6]:
data["src_port"] = data["src_port"].apply(str)
data["dst_port"] = data["dst_port"].apply(str)

In [7]:
train, validate, test = np.split(data, [int(.7*len(data)), int(.85*len(data))])
train = train.sample(frac=1)
validate = validate.sample(frac=1)
test = test.sample(frac=1)
y_train = train.loc[:, "attack_flag"].values.ravel()
X_train = train.drop(["attack_flag"], axis=1)
y_validate = validate.loc[:, "attack_flag"].values.ravel()
X_validate = validate.drop(["attack_flag"], axis=1)
y_test = test.loc[:, "attack_flag"].values.ravel()
X_test = test.drop(["attack_flag"], axis=1)
print(train.shape)
print(validate.shape)
print(test.shape)

(152339, 27)
(32644, 27)
(32645, 27)


In [8]:
deep_numeric_feat = ["Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "well_known_src_pool", "registered_src_pool", "cli_src_pool"]
wide_numeric_feat = ["conn_len",  "src_bytes", "dst_bytes", "conn_numb"]
categorical_feat = ["service", "Conn_state", "src_port", "dst_port", "proto"]

In [ ]:
deep_numeric_feat_tf = [tf.feature_column.numeric_column(k) for k in deep_numeric_feat]
wide_numeric_feat_tf = [tf.feature_column.numeric_column(k) for k in wide_numeric_feat]

In [ ]:
#data["src_port"].unique().tolist()

In [ ]:
src_port_mod = ["src_" + s for s in data["src_port"].unique().tolist()]
src_port = tf.feature_column.categorical_column_with_vocabulary_list(
    'src_port', src_port_mod)

dst_port_mod = ["dst_" + s for s in data["dst_port"].unique().tolist()]
dst_port = tf.feature_column.categorical_column_with_vocabulary_list(
    'dst_port', dst_port_mod)

service = tf.feature_column.categorical_column_with_vocabulary_list(
    'service', data["service"].unique().tolist())

Conn_state = tf.feature_column.categorical_column_with_vocabulary_list(
    'Conn_state', data["Conn_state"].unique().tolist())

proto = tf.feature_column.categorical_column_with_vocabulary_list(
    'proto', data["proto"].unique().tolist())

In [ ]:
deep_columns = [
    tf.feature_column.indicator_column(service),
    tf.feature_column.indicator_column(Conn_state),
    tf.feature_column.indicator_column(proto),
    tf.feature_column.embedding_column(src_port, dimension=150),
    tf.feature_column.embedding_column(dst_port, dimension=100),
]

In [ ]:
crossed_columns = [
      tf.feature_column.crossed_column(
          ['src_port', 'service'], hash_bucket_size=10000),
      tf.feature_column.crossed_column(
          ['dst_port', 'service'], hash_bucket_size=10000),
      tf.feature_column.crossed_column(
          ['conn_len', 'src_bytes'], hash_bucket_size=10000),
      tf.feature_column.crossed_column(
          ['conn_len', 'dst_bytes'], hash_bucket_size=10000),
      tf.feature_column.crossed_column(
          ['dst_port', 'conn_numb'], hash_bucket_size=10000),  
      tf.feature_column.crossed_column(
          ['service', 'proto'], hash_bucket_size=100),
    
  ]

In [ ]:
model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir = "/home/matz/Desktop/ml_ids/NN_model2",
    dnn_activation_fn=tf.nn.selu,
    linear_feature_columns = deep_columns + deep_numeric_feat_tf + wide_numeric_feat_tf + crossed_columns,
    dnn_feature_columns = deep_columns + deep_numeric_feat_tf + wide_numeric_feat_tf,
    dnn_hidden_units = [750 ,550, 250, 125, 75, 55, 35, 20]
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/matz/Desktop/ml_ids/NN_model2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa2df340630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
feature_set= deep_numeric_feat + wide_numeric_feat + categorical_feat

In [ ]:
def get_input_fn(data_set, num_epochs=None, n_batch = 256, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
       x=pd.DataFrame({k: data_set[k].values for k in feature_set}),
       y = pd.Series(data_set["attack_flag"].values),
       batch_size=n_batch,   
       num_epochs=num_epochs,
       shuffle=shuffle)

In [ ]:
model.train(input_fn=get_input_fn(train, n_batch=180), steps=20000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /home/matz/Desktop/ml_ids/NN_model2/model.ckpt.
INFO:tensorflow:loss = 392.48584, step = 1
INFO:tensorflow:global_step/sec: 57.8409
INFO:tensorflow:loss = 12.114062, step = 101 (1.730 sec)
INFO:tensorflow:global_step/sec: 70.2605
INFO:tensorflow:loss = 4.3849106, step = 201 (1.423 sec)
INFO:tensorflow:global_step/sec: 71.792
INFO:tensorflow:loss = 3.9444952, step = 3

INFO:tensorflow:global_step/sec: 70.418
INFO:tensorflow:loss = 6.723247, step = 6701 (1.420 sec)
INFO:tensorflow:global_step/sec: 71.6994
INFO:tensorflow:loss = 1.7089075, step = 6801 (1.395 sec)
INFO:tensorflow:global_step/sec: 70.5313
INFO:tensorflow:loss = 0.5356589, step = 6901 (1.418 sec)
INFO:tensorflow:global_step/sec: 71.3952
INFO:tensorflow:loss = 0.94464684, step = 7001 (1.400 sec)
INFO:tensorflow:global_step/sec: 70.7504
INFO:tensorflow:loss = 1.4725621, step = 7101 (1.413 sec)
INFO:tensorflow:global_step/sec: 71.2161
INFO:tensorflow:loss = 9.368528, step = 7201 (1.404 sec)
INFO:tensorflow:global_step/sec: 71.2466
INFO:tensorflow:loss = 1.9766287, step = 7301 (1.403 sec)
INFO:tensorflow:global_step/sec: 67.7669
INFO:tensorflow:loss = 3.1579757, step = 7401 (1.476 sec)
INFO:tensorflow:global_step/sec: 69.3492
INFO:tensorflow:loss = 6.4788713, step = 7501 (1.442 sec)
INFO:tensorflow:global_step/sec: 76.697
INFO:tensorflow:loss = 2.2219017, step = 7601 (1.304 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 65.1536
INFO:tensorflow:loss = 1.7885835, step = 15001 (1.535 sec)
INFO:tensorflow:global_step/sec: 58.1144
INFO:tensorflow:loss = 2.255279, step = 15101 (1.721 sec)
INFO:tensorflow:global_step/sec: 59.0421
INFO:tensorflow:loss = 2.3941216, step = 15201 (1.694 sec)
INFO:tensorflow:global_step/sec: 43.5259
INFO:tensorflow:loss = 2.2476594, step = 15301 (2.297 sec)
INFO:tensorflow:global_step/sec: 59.3091
INFO:tensorflow:loss = 2.0672674, step = 15401 (1.685 sec)
INFO:tensorflow:global_step/sec: 55.563
INFO:tensorflow:loss = 3.7691987, step = 15501 (1.800 sec)
INFO:tensorflow:global_step/sec: 52.2497
INFO:tensorflow:loss = 1.9994286, step = 15601 (1.913 sec)
INFO:tensorflow:global_step/sec: 53.1851
INFO:tensorflow:loss = 1.1855084, step = 15701 (1.881 sec)
INFO:tensorflow:global_step/sec: 56.0877
INFO:tensorflow:loss = 0.65526146, step = 15801 (1.783 sec)
INFO:tensorflow:global_step/sec: 67.8255
INFO:tensorflow:loss = 1.5324699, step = 15901 (1.474 sec)
I

In [ ]:
eval_metrics = model.evaluate(input_fn=get_input_fn(validate, n_batch=validate.shape[0], shuffle=False), steps=1)

In [ ]:
for key,value in sorted(eval_metrics.items()):
  print('%s: %s' % (key, value))

In [ ]:
results = []
results = list(model.predict(input_fn=get_input_fn(test, num_epochs=1, n_batch = X_test.shape[0], shuffle=False)))

In [ ]:
y_pred = []
for el in results:
    y_pred.append(el["class_ids"])

In [ ]:
name = "NN"

Acc = {}
F1S = {}
Prec = {}
Rec = {}
FPR = {}

Acc[name] = metrics.accuracy_score(test["attack_flag"], y_pred)
F1S[name] = metrics.f1_score(test["attack_flag"], y_pred)
Prec[name] = metrics.precision_score(test["attack_flag"], y_pred)
Rec[name] = metrics.recall_score(test["attack_flag"], y_pred)
tn, fp, fn, tp = metrics.confusion_matrix(test["attack_flag"], y_pred).ravel()
FPR[name] = fp/(fp+tn)

print("{0:2} Accuracy: {1:.5f}, F1-score: {2:.5f}, Precision: {3:.5f}, Recall: {4:.5f}, FPR: {5:.5f}".format(name, Acc[name], F1S[name], Prec[name], Rec[name], FPR[name]))
print("TN: {0:3}; FP: {1:3}; FN: {2:4}; TP: {3:3}\n".format(tn, fp, fn, tp))

## New data prediction

In [286]:
test_set = pd.read_csv("20151220.txt", sep = "\t", header = None)
test_set.columns = ["conn_len", "service", "src_bytes", "dst_bytes", "conn_numb", "Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "Conn_state", "IDS_detection", "Malware_detection", "Ashula_detection", "attack_flag", "src_IP", "src_port", "dst_IP", "dst_port", "start_time", "proto"]
print(Counter(test_set["attack_flag"]))

Counter({-1: 189082, 1: 12436})


In [65]:
test_set.loc[:, "attack_flag"] = test_set.loc[:,"attack_flag"].replace(-2, -1)
test_set.loc[:, "attack_flag"] = test_set.loc[:, "attack_flag"] * -1
test_set.loc[:, "attack_flag"] = test_set.loc[:,"attack_flag"].replace(-1, 0)

In [27]:
# removing NaNs
test_set = test_set.dropna()
test_set.shape

(201518, 24)

In [28]:
test_set["well_known_src_pool"] = test_set.apply(wk, axis=1)
test_set["registered_src_pool"] = test_set.apply(reg, axis=1)
test_set["cli_src_pool"] = test_set.apply(cli, axis=1)

In [29]:
test_set["src_port"] = test_set["src_port"].apply(str)
test_set["dst_port"] = test_set["dst_port"].apply(str)

In [30]:
#test_new_data = test_set.sample(n=30000, random_state=1)
test_new_data = test_set.loc[range(30000), :]
test_new_data = test_new_data.sample(frac=1)
y_test_new_data = test_new_data.loc[:, "attack_flag"].values.ravel()
X_test_new_data = test_new_data.drop(["attack_flag"], axis=1)
print(test_new_data.shape)

(30000, 27)


In [31]:
print(Counter(test_new_data["attack_flag"]))

Counter({1: 29540, 0: 460})


In [32]:
results = []
results = list(model.predict(input_fn=get_input_fn(test_new_data, num_epochs=1, n_batch = X_test.shape[0], shuffle=False)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/matz/Desktop/ml_ids/NN_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [33]:
y_pred = []
for el in results:
    y_pred.append(el["class_ids"])

In [34]:
name = "NN"

Acc = {}
F1S = {}
Prec = {}
Rec = {}
FPR = {}

Acc[name] = metrics.accuracy_score(y_test_new_data, y_pred)
F1S[name] = metrics.f1_score(y_test_new_data, y_pred)
Prec[name] = metrics.precision_score(y_test_new_data, y_pred)
Rec[name] = metrics.recall_score(y_test_new_data, y_pred)
tn, fp, fn, tp = metrics.confusion_matrix(y_test_new_data, y_pred).ravel()
FPR[name] = fp/(fp+tn)

print("{0:2} Accuracy: {1:.5f}, F1-score: {2:.5f}, Precision: {3:.5f}, Recall: {4:.5f}, FPR: {5:.5f}".format(name, Acc[name], F1S[name], Prec[name], Rec[name], FPR[name]))
print("TN: {0:3}; FP: {1:3}; FN: {2:4}; TP: {3:3}\n".format(tn, fp, fn, tp))

NN Accuracy: 0.98410, F1-score: 0.99192, Precision: 0.99284, Recall: 0.99100, FPR: 0.45870
TN: 249; FP: 211; FN:  266; TP: 29274

